In [8]:
#imprt necessary libaries and modules
import pandas as pd
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20
from  sklearn.model_selection import train_test_split
import numpy as np
import csv
import category_encoders as ce
from numpy.random import RandomState


In [4]:
# read kidney disease data file, remove first two rows
data =  pd.read_csv("/Users/Morgan/Documents/316/ckd-dataset-v2.csv")
kidneydisease = data.iloc[2:]
kidneyheads = kidneydisease.head(20)

In [44]:
#Find unique values for each attributes and values
kidneydisease['age'].value_counts()
kidneydisease['sg'].value_counts()
kidneydisease["grf"].value_counts()
kidneydisease["al"].value_counts()
kidneydisease["su"].value_counts()
kidneydisease["bgr"].value_counts()
kidneydisease["bu"].value_counts()
kidneydisease["sod"].value_counts()
kidneydisease["sc"].value_counts()
kidneydisease["pot"].value_counts()
kidneydisease["hemo"].value_counts()
kidneydisease["pcv"].value_counts()
kidneydisease["rbcc"].value_counts()
kidneydisease["wbcc"].value_counts()

7360 - 9740      98
4980 - 7360      47
9740 - 12120     29
< 4980           10
12120 - 14500     6
14500 - 16880     6
16880 - 19260     2
≥ 24020           1
19260 - 21640     1
Name: wbcc, dtype: int64

In [45]:
#create dictionary for the purpose of ordinal encoding
dic = [{'col' : 'age', 'mapping' :{'< 12': 1, '12 - 20 ':2, '20 - 27':3, '27 - 35:':4, '35 - 43': 5, '43 - 51' : 6,
                                    '51 - 59': 7, '59 - 66 ': 8, '66 - 74': 9, '≥ 74': 10}}, 
       {'col' : 'sg', 'mapping' :{'< 1.007': 1, '1.009 - 1.011':2, '1.015 - 1.017':3, '1.019 - 1.021':4}},
       {'col' : 'al', 'mapping' :{'0': 1, '1 - 1':2, '2 - 2':2, '3 - 3':3, '≥ 4': 4}},
       {'col' : 'grf', 'mapping' :{'< 26.6175': 1, '26.6175 - 51.7832':2, '51.7832 - 76.949':3, '76.949 - 102.115':4, 
                                   '102.115 - 127.281': 5, '127.281 - 152.446 ':6, '152.446 - 177.612 ':7,
                                    '177.612 - 202.778': 8, '202.778 - 227.944': 9, '≥ 227.944': 10, 'p':11}},
      {'col' : 'su', 'mapping' :{'< 0': 1, '2 - 2':2, '3 - 4':3, '4 - 4':4, 
                                   '≥ 4': 5 }},
      {'col' : 'bgr', 'mapping' :{'< 112': 1, '112 - 154':2, '154 - 196':3, '196 - 238':4, 
                                   '238 - 280': 5,  '280 - 322': 6, '322 - 364': 7, '364 - 406': 8, '406 - 448': 9, '≥ 448': 10}},
      {'col' : 'bu', 'mapping' :{'< 48.1': 1, '48.1 - 86.2':2, '86.2 - 124.3':3, '124.3 - 162.4':4, 
                                   '162.4 - 200.5': 5,  '200.5 - 238.6': 6, '238.6 - 276.7': 7, '≥ 352.9': 8, '406 - 448': 9, '≥ 448': 10}},
      {'col' : 'sod', 'mapping' :{'< 118': 1, '118 - 123':2, '123 - 128':3, '128 - 133':4, 
                                   '133 - 138': 5,  '138 - 143': 6, '143 - 148': 7, '148 - 153': 8, '406 - 448': 9, '≥ 158': 10}},
      {'col' : 'sc', 'mapping' :{'< 3.65': 1, '3.65 - 6.8':2, '6.8 - 9.95':3, '9.95 - 13.1':4, 
                                   '13.1 - 16.25': 5,  '16.25 - 19.4': 6, '≥ 28.85 ': 7}},
      {'col' : 'pot', 'mapping' :{'< 7.31': 1, '7.31 - 11.72':2, '38.18 - 42.59':3, '≥ 42.59':4, }},
      {'col' : 'hemo', 'mapping' :{'< 6.1': 1, '6.1 - 7.4':2, '7.4 - 8.7':3, '8.7 - 10':4, 
                                   '10 - 11.3': 5,  '11.3 - 12.6': 6, '12.6 - 13.9': 7, '13.9 - 15.2': 8, '15.2 - 16.5': 9, '≥ 16.5': 10}},
      {'col' : 'pcv', 'mapping' :{'< 17.9': 1, '17.9 - 21.8':2, '21.8 - 25.7':3, '25.7 - 29.6':4, 
                                   '29.6 - 33.5': 5,  '33.5 - 37.4': 6, '37.4 - 41.3': 7, '41.3 - 45.2': 8, '45.2 - 49.1': 9, '≥ 49.1': 10}},
      {'col' : 'rbcc', 'mapping' :{'< 2.69': 1, '2.69 - 3.28':2, '3.28 - 3.87':3, '3.87 - 4.46':4, 
                                   '4.46 - 5.05': 5,  '5.05 - 5.64': 6, '5.64 - 6.23': 7, '6.23 - 6.82': 8, '≥ 7.41': 9}},
      {'col' : 'wbcc', 'mapping' :{'< 4980 ': 1, '4980 - 7360':2, '7360 - 9740':3, '9740 - 12120':4, 
                                   '12120 - 14500': 5,  '14500 - 16880': 6, '16880 - 19260': 7, '19260 - 21640': 8, '≥ 24020': 9}}]

In [46]:
#instantiate encoder, using the dictionary created previously
ordinal = ce.OrdinalEncoder(cols = ['age', 'sg', 'al', 'grf', 'su',
                                    'bgr', 'bu', 'sod', 'sc', 'pot', 'hemo', 'pcv', 'rbcc', 'wbcc'], mapping = dic)

rng = RandomState()
#Split into traing and test set ( 0.7 , 0.3)
x_train = kidneydisease.sample(frac=0.7, random_state=rng)
x_test = kidneydisease.loc[~kidneydisease.index.isin(x_train.index)]

In [47]:
#Encode training set using ordinal encoding library function
x_train_encoded = ordinal.fit_transform(x_train)

In [48]:
x_train_encoded.head(10)

,bp (Diastolic),bp limit,sg,al,class,rbc,su,pc,pcc,ba,...,htn,dm,cad,appet,pe,ane,grf,stage,affected,age
30,0,0,-1.0,-1.0,notckd,0,1.0,0,0,0,...,0,0,0,0,0,0,8.0,s1,0,-1.0
148,1,1,2.0,3.0,ckd,0,3.0,0,0,0,...,1,1,1,0,0,0,1.0,s5,1,-1.0
49,1,2,2.0,2.0,ckd,0,1.0,0,0,0,...,0,0,0,0,0,0,4.0,s2,1,6.0
69,1,1,-1.0,-1.0,notckd,0,1.0,0,0,0,...,0,0,0,0,0,0,5.0,s1,0,6.0
93,0,0,4.0,-1.0,ckd,0,1.0,0,0,0,...,0,0,0,0,0,0,1.0,s5,1,7.0
84,1,2,3.0,4.0,ckd,0,3.0,1,0,0,...,1,1,0,0,0,1,1.0,s5,1,7.0
45,1,2,4.0,2.0,ckd,1,1.0,1,1,0,...,0,0,0,1,1,1,2.0,s4,1,5.0
47,1,1,4.0,-1.0,notckd,0,1.0,0,0,0,...,0,0,0,0,0,0,4.0,s2,0,6.0
57,0,0,-1.0,-1.0,notckd,0,1.0,0,0,0,...,0,0,0,0,0,0,4.0,s2,0,6.0
177,0,0,3.0,4.0,ckd,0,1.0,0,0,0,...,1,1,0,1,1,0,1.0,s5,1,9.0


In [49]:
x_test.head()

,bp (Diastolic),bp limit,sg,al,class,rbc,su,pc,pcc,ba,...,htn,dm,cad,appet,pe,ane,grf,stage,affected,age
2,0,0,1.019 - 1.021,1 - 1,ckd,0,< 0,0,0,0,...,0,0,0,0,0,0,≥ 227.944,s1,1,< 12
3,0,0,1.009 - 1.011,< 0,ckd,0,< 0,0,0,0,...,0,0,0,0,0,0,≥ 227.944,s1,1,< 12
4,0,0,1.009 - 1.011,≥ 4,ckd,1,< 0,1,0,1,...,0,0,0,1,0,0,127.281 - 152.446,s1,1,< 12
13,0,0,≥ 1.023,< 0,notckd,0,< 0,0,0,0,...,0,0,0,0,0,0,152.446 - 177.612,s1,0,20 - 27
15,0,0,1.015 - 1.017,2 - 2,ckd,0,4 - 4,1,0,0,...,0,1,0,0,1,0,76.949 - 102.115,s2,1,20 - 27


In [50]:
#Encode testing set using ordinal encoding library function
x_test_encoded = ordinal.transform(x_test)

In [51]:
x_test_encoded.head()

,bp (Diastolic),bp limit,sg,al,class,rbc,su,pc,pcc,ba,...,htn,dm,cad,appet,pe,ane,grf,stage,affected,age
2,0,0,4.0,2.0,ckd,0,1.0,0,0,0,...,0,0,0,0,0,0,10.0,s1,1,1.0
3,0,0,2.0,-1.0,ckd,0,1.0,0,0,0,...,0,0,0,0,0,0,10.0,s1,1,1.0
4,0,0,2.0,4.0,ckd,1,1.0,1,0,1,...,0,0,0,1,0,0,-1.0,s1,1,1.0
13,0,0,-1.0,-1.0,notckd,0,1.0,0,0,0,...,0,0,0,0,0,0,-1.0,s1,0,3.0
15,0,0,3.0,2.0,ckd,0,4.0,1,0,0,...,0,1,0,0,1,0,4.0,s2,1,3.0


In [52]:
# see attributes property after ordinal encoding
x_train_encoded.dtypes

bp (Diastolic)     object
bp limit           object
sg                float64
al                float64
class              object
rbc                object
su                float64
pc                 object
pcc                object
ba                 object
bgr                 int32
bu                  int32
sod                 int32
sc                float64
pot                 int32
hemo                int32
pcv                 int32
rbcc                int32
wbcc              float64
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
grf               float64
stage              object
affected           object
age               float64
dtype: object

In [53]:
#one hot encoding using onehotencoder()
al_values = x_train_encoded[["stage"]]
OneHot = OneHotEncoder()
One_Hot_stage= OneHot.fit_transform(al_values)
print(One_Hot_stage)
#results of one hot encoding

  (0, 0)	1.0
  (1, 4)	1.0
  (2, 1)	1.0
  (3, 0)	1.0
  (4, 4)	1.0
  (5, 4)	1.0
  (6, 3)	1.0
  (7, 1)	1.0
  (8, 1)	1.0
  (9, 4)	1.0
  (10, 4)	1.0
  (11, 0)	1.0
  (12, 2)	1.0
  (13, 3)	1.0
  (14, 4)	1.0
  (15, 3)	1.0
  (16, 4)	1.0
  (17, 0)	1.0
  (18, 2)	1.0
  (19, 3)	1.0
  (20, 3)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 2)	1.0
  (24, 4)	1.0
  :	:
  (115, 3)	1.0
  (116, 3)	1.0
  (117, 0)	1.0
  (118, 4)	1.0
  (119, 0)	1.0
  (120, 3)	1.0
  (121, 0)	1.0
  (122, 1)	1.0
  (123, 0)	1.0
  (124, 1)	1.0
  (125, 0)	1.0
  (126, 3)	1.0
  (127, 4)	1.0
  (128, 0)	1.0
  (129, 2)	1.0
  (130, 0)	1.0
  (131, 1)	1.0
  (132, 4)	1.0
  (133, 1)	1.0
  (134, 3)	1.0
  (135, 2)	1.0
  (136, 2)	1.0
  (137, 2)	1.0
  (138, 3)	1.0
  (139, 1)	1.0


In [54]:
#results of one hot encoding
One_hot_stage.toarray()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1.

In [55]:
#Table Correlation between different attribute in different columns 
x_train_encoded.corr()

,sg,al,su,bgr,bu,sod,sc,pot,hemo,pcv,rbcc,wbcc,grf,age
sg,1.000000,0.074010,-0.008568,0.073552,0.133558,0.077397,0.081926,0.013833,-0.297024,-0.270816,-0.092491,0.124538,-0.022463,0.089348
al,0.074010,1.000000,0.232364,0.373454,0.359596,-0.295220,0.252469,0.194493,-0.462253,-0.443617,-0.448288,0.285167,-0.223958,-0.047898
su,-0.008568,0.232364,1.000000,0.411541,0.123656,-0.032339,-0.052503,0.285646,-0.070174,-0.068596,-0.126531,0.055712,-0.066352,0.069298
bgr,0.073552,0.373454,0.411541,1.000000,0.118188,-0.196096,0.048087,0.072646,-0.242776,-0.224213,-0.271785,0.087239,-0.160149,-0.042516
bu,0.133558,0.359596,0.123656,0.118188,1.000000,-0.267324,0.716101,0.388453,-0.526617,-0.510069,-0.446968,0.106352,-0.343587,0.181938
sod,0.077397,-0.295220,-0.032339,-0.196096,-0.267324,1.000000,-0.495560,0.148900,0.460693,0.468068,0.469130,0.090555,0.206971,0.098112
sc,0.081926,0.252469,-0.052503,0.048087,0.716101,-0.495560,1.000000,0.014903,-0.502667,-0.506371,-0.415493,-0.016643,-0.242589,0.043781
pot,0.013833,0.194493,0.285646,0.072646,0.388453,0.148900,0.014903,1.000000,-0.144402,-0.136908,-0.185197,-0.094249,-0.077882,0.059972
hemo,-0.297024,-0.462253,-0.070174,-0.242776,-0.526617,0.460693,-0.502667,-0.144402,1.000000,0.856412,0.637303,-0.170458,0.268178,-0.023999
pcv,-0.270816,-0.443617,-0.068596,-0.224213,-0.510069,0.468068,-0.506371,-0.136908,0.856412,1.000000,0.655073,-0.161182,0.306370,-0.066203


In [59]:
#Rank and correlation between class columns and different selected attributes
#using spearman and kendall methods

x_train_encoded['class'].corr(x_train_encoded['age'], method= 'spearman')

-0.06881422118818212

In [60]:
x_train_encoded['class'].corr(x_train_encoded['al'], method= 'kendall')

-0.566485363539907

In [61]:
x_train_encoded['class'].corr(x_train_encoded['sg'], method='spearman')

-0.10686455250164387

In [62]:
x_train_encoded['class'].corr(x_train_encoded['grf'], method='kendall')

0.28985873714617694

In [21]:
x_train_encoded['class'].corr(x_train_encoded['su'], method='spearman')

-0.23300299450299464